# Sentinel-1 SLC preprocessing

This notebook contain an example on how to execute the Sentinel-1 SLC preprocessing on a local machine with SNAP installed. This workflow can produce a stack of coregistered Sentinel-1 SLC bursts

In [1]:
import urllib
import json
from IPython.display import JSON

from datetime import datetime

import glob
import os
import subprocess
import shutil

## Input definition

In [2]:
burstId = 249435
sub_swath = 'IW2'
polarization = 'VV'
start_date = '2024-08-01'
end_date = '2024-09-30'
mst_date = '20240809'

In [3]:
graph_mst_filename = '../graphs/pre-processing_2images_SaveMst_GeoTiff.xml'
graph_slv_filename = '../graphs/pre-processing_2images_SaveOnlySlv_GeoTiff.xml'

In [4]:
if os.path.exists("../CDSE_SECRET"):
    with open("../CDSE_SECRET", "r") as file:
        lines = file.readlines()
    CDSE_ACCESS_KEY = lines[0].strip().split(": ")[1]
    CDSE_SECRET_KEY = lines[1].strip().split(": ")[1]
else:
    # get from environment vatiables:
    CDSE_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID", None)
    CDSE_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)


output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing'
if not os.path.exists(output_folder):
    output_folder = os.path.abspath("output")
print("output_folder: " + str(output_folder))

output_folder: /home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing


## Burst download

In [5]:
https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
    f"PolarisationChannels eq '{polarization}' and "
    f"BurstId eq {burstId} and "
    f"SwathIdentifier eq '{sub_swath}'"
) + "&$top=1000"

with urllib.request.urlopen(https_request) as response:
    content = response.read().decode()
bursts = json.loads(content)
JSON(bursts)

<IPython.core.display.JSON object>

In [6]:
for b in bursts['value']:
    
    burst_extract_cmd = (
        f"docker run -v {output_folder}:/home/ubuntu "
        f"-e AWS_ACCESS_KEY_ID={CDSE_ACCESS_KEY} -e AWS_SECRET_ACCESS_KEY={CDSE_SECRET_KEY} "
        f"cdse_utilities sentinel1_burst_extractor.sh -o /home/ubuntu "
        f"-n {b['ParentProductName']} -p {polarization.lower()} -s {sub_swath.lower()} -r {burstId}"
    )

    #print(burst_extract_cmd)
    os.system(burst_extract_cmd)

Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26039, 1510
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.


## Sentinel-1 SLC preprocessing

Input preparation

In [7]:
mst_filename = glob.glob(os.path.join(output_folder, f'*{mst_date}*', 'manifest.safe'))[0]

slv_filename_list = glob.glob(os.path.join(output_folder, f'*SAFE', 'manifest.safe'))
slv_filename_list.remove(mst_filename)

Run the first preprocessing saving the master with the associated lat/lon grid

In [8]:
mst_bandname = f'{sub_swath}_VV_mst_{datetime.strptime(mst_date, "%Y%m%d").strftime("%d%b%Y")}'

slv_filename = slv_filename_list[0]
slv_date = os.path.basename(os.path.dirname(slv_filename)).split('_')[2][:8]
slv_bandname = f'{sub_swath}_VV_slv1_{datetime.strptime(slv_date, "%Y%m%d").strftime("%d%b%Y")}'

output_mst_filename = os.path.join(output_folder, f'S1_slc_{mst_date}.tif')
output_slv_filename = os.path.join(output_folder, f'S1_slc_{slv_date}.tif')

cmd = (
    f'gpt "{graph_mst_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" '
    f'-Pi_q_mst_bandnames="i_{mst_bandname},q_{mst_bandname}" '
    f'-Pi_q_slv_bandnames="i_{slv_bandname},q_{slv_bandname}" '
    f'-Poutput_mst_filename="{output_mst_filename}" -Poutput_slv_filename="{output_slv_filename}" -Ppolarisation="VV"'
)

print(cmd)
os.system(cmd)

gpt "../graphs/pre-processing_2images_SaveMst_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1A_SLC_20240821T170739_249435_IW2_VV_442005.SAFE/manifest.safe" -Pi_q_mst_bandnames="i_IW2_VV_mst_09Aug2024,q_IW2_VV_mst_09Aug2024" -Pi_q_slv_bandnames="i_IW2_VV_slv1_21Aug2024,q_IW2_VV_slv1_21Aug2024" -Poutput_mst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1_slc_20240809.tif" -Poutput_slv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1_slc_20240821.tif" -Ppolarisation="VV"


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph


1383 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
1384 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
1387 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
1387 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.
INFO: eu.esa.sar.commons.io.ImageIOFile: Using FileCacheImageInputStream


....10%....20%....30%....40%....50%....60%....70%....80%....90% done.


-- org.jblas INFO Deleting /tmp/jblas2117985447933282453/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas2117985447933282453/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas2117985447933282453/libgfortran-5.so
-- org.jblas INFO Deleting /tmp/jblas2117985447933282453/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas2117985447933282453


0

Run the preprocessing for the remaining slave images

In [9]:
for slv_filename in slv_filename_list[1:]:

    slv_date = os.path.basename(os.path.dirname(slv_filename)).split('_')[2][:8]
    slv_bandname = f'{sub_swath}_VV_slv1_{datetime.strptime(slv_date, "%Y%m%d").strftime("%d%b%Y")}'
    output_slv_filename = os.path.join(output_folder, f'S1_slc_{slv_date}.tif')

    cmd = (
        f'gpt "{graph_slv_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" '
        f'-Pi_q_slv_bandnames="i_{slv_bandname},q_{slv_bandname}" '
        f'-Poutput_slv_filename="{output_slv_filename}" -Ppolarisation="VV"'
    )
    
    print(cmd)
    os.system(cmd)

gpt "../graphs/pre-processing_2images_SaveOnlySlv_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1A_SLC_20240914T170740_249435_IW2_VV_445439.SAFE/manifest.safe" -Pi_q_slv_bandnames="i_IW2_VV_slv1_14Sep2024,q_IW2_VV_slv1_14Sep2024" -Poutput_slv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/SLC_preprocessing/S1_slc_20240914.tif" -Ppolarisation="VV"


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph


1769 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
1769 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
1774 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
1774 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.
INFO: eu.esa.sar.commons.io.ImageIOFile: Using FileCacheImageInputStream


....10%....20%....30%....40%....50%....60%....70%....80%....90% done.


...

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph


1143 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
1143 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
1146 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
1146 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.


....10%....20%....30%....40%....50%....60%....70%....80%....90% done.


-- org.jblas INFO Deleting /tmp/jblas1759550498567934752/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas1759550498567934752/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas1759550498567934752/libgfortran-5.so
-- org.jblas INFO Deleting /tmp/jblas1759550498567934752/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas1759550498567934752
